In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("dataset.csv")
# Display the first few rows of the DataFrame
print(df.head())

           frame_path  up_state  up_hold_duration  left_state   
0  frames/frame_0.jpg         0               0.0           0  \
1  frames/frame_1.jpg         0               0.0           0   
2  frames/frame_2.jpg         0               0.0           0   
3  frames/frame_3.jpg         0               0.0           0   
4  frames/frame_4.jpg         0               0.0           0   

   left_hold_duration  right_state  right_hold_duration  
0                 0.0            0                  0.0  
1                 0.0            0                  0.0  
2                 0.0            0                  0.0  
3                 0.0            0                  0.0  
4                 0.0            0                  0.0  


In [3]:
# Load the image data

from PIL import Image
import numpy as np
df['frame_path'] = df['frame_path'].apply(lambda x: np.array(Image.open(x)))

In [4]:
# normalize the image data
df['frame_path'] = df['frame_path'].apply(lambda x: x/255)

In [5]:
# Columns to be dropped
drop_columns = [
                'up_hold_duration',
                'left_hold_duration',
                'right_hold_duration']

# Drop the columns
df = df.drop(columns=drop_columns)

print(df.head())


                                          frame_path  up_state  left_state   
0  [[[0.9764705882352941, 0.9686274509803922, 0.9...         0           0  \
1  [[[0.0196078431372549, 0.0196078431372549, 0.0...         0           0   
2  [[[0.9882352941176471, 0.984313725490196, 0.96...         0           0   
3  [[[0.00784313725490196, 0.027450980392156862, ...         0           0   
4  [[[0.0196078431372549, 0.03137254901960784, 0....         0           0   

   right_state  
0            0  
1            0  
2            0  
3            0  
4            0  


In [6]:
# # Create a new column 'empty_state' initialized with zeros
# df['empty_state'] = 0

# # Iterate through rows
# for i in range(len(df)):
#     # If 'left' and 'right' are the same
#     if df.loc[i, 'left_state'] == df.loc[i, 'right_state']:
#         # Replace 'left' and 'right' with 0
#         df.loc[i, 'left_state'] = 0
#         df.loc[i, 'left_state'] = 0
#         # Assign 'empty_state' to 1
#         df.loc[i, 'empty_state'] = 1
    
#     # else assign 'empty_state' to 0
#     else:
#         df.loc[i, 'empty_state'] = 0

# # Display the first few rows of the DataFrame
# print(df.head())

In [7]:
df.describe()

,up_state,left_state,right_state
count,11933.000000,11933.000000,11933.000000
mean,0.692198,0.023800,0.158803
std,0.461603,0.152431,0.365508
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000


In [8]:
# itterate through rows and drop the rows where up, left and right are all 0
for i in range(len(df)):
    if df.loc[i, 'up_state'] == 0 and df.loc[i, 'left_state'] == 0 and df.loc[i, 'right_state'] == 0:
        df = df.drop(i)

# Display the first few rows of the DataFrame
print(df.head())


                                            frame_path  up_state  left_state   
525  [[[0.9529411764705882, 0.9568627450980393, 0.9...         1           0  \
526  [[[0.9568627450980393, 0.9372549019607843, 0.9...         1           0   
527  [[[0.9529411764705882, 0.9568627450980393, 0.9...         1           0   
528  [[[0.9529411764705882, 0.9568627450980393, 0.9...         1           0   
529  [[[0.9372549019607843, 0.9607843137254902, 0.9...         1           0   

     right_state  
525            0  
526            0  
527            0  
528            0  
529            0  


In [9]:
# drop up_state column
df = df.drop(columns=['up_state'])

In [10]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.2,random_state=42, shuffle=True)
# check train minotiry class
train_data.describe()

,left_state,right_state
count,6734.000000,6734.000000
mean,0.033858,0.225720
std,0.180877,0.418087
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,1.000000,1.000000


In [11]:
# Define features (X) and labels (Y) for training set
X_train = train_data['frame_path']
Y_train = train_data.drop('frame_path', axis=1)

# Define features (X) and labels (Y) for testing set
X_test = test_data['frame_path']
Y_test = test_data.drop('frame_path', axis=1)

In [12]:
X_train = np.stack(train_data['frame_path'])
X_test = np.stack(test_data['frame_path'])

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Define the model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1:])),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Flatten(),    # Flattening the 3D outputs of the previous layer

    Dense(512, activation='relu'),
    Dropout(0.5),

    Dense(256, activation='relu'),
    Dropout(0.5),


    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(Y_train.shape[1], activation='sigmoid')   # number of output neurons equals to number of key states
])



In [17]:

# Compile the model
model.compile(optimizer='adam', 
              loss='binary_crossentropy',   # use binary_crossentropy for multi-label classification
              metrics=['accuracy'])

# Print a summary of the model's architecture
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 238, 318, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 119, 159, 32)     0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 119, 159, 32)      0         
                                                                 
 conv2d_4 (Conv2D)           (None, 117, 157, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 58, 78, 64)       0         
 2D)                                                             
                                                                 
 dropout_7 (Dropout)         (None, 58, 78, 64)       

In [18]:
# Train the model
history = model.fit(X_train, Y_train, epochs=9, batch_size=16, validation_split=0.2)

Epoch 1/9
337/337 [==============================] - 12s 32ms/step - loss: 0.3842 - accuracy: 0.2304 - val_loss: 0.3582 - val_accuracy: 0.2413
Epoch 2/9
337/337 [==============================] - 9s 27ms/step - loss: 0.3144 - accuracy: 0.3263 - val_loss: 0.2372 - val_accuracy: 0.5249
Epoch 3/9
337/337 [==============================] - 9s 27ms/step - loss: 0.2236 - accuracy: 0.5591 - val_loss: 0.1778 - val_accuracy: 0.5813
Epoch 4/9
337/337 [==============================] - 9s 26ms/step - loss: 0.1772 - accuracy: 0.6859 - val_loss: 0.1637 - val_accuracy: 0.8597
Epoch 5/9
337/337 [==============================] - 9s 27ms/step - loss: 0.1590 - accuracy: 0.7147 - val_loss: 0.1435 - val_accuracy: 0.8441
Epoch 6/9
337/337 [==============================] - 9s 26ms/step - loss: 0.1437 - accuracy: 0.7602 - val_loss: 0.1393 - val_accuracy: 0.8330
Epoch 7/9
337/337 [==============================] - 9s 26ms/step - loss: 0.1405 - accuracy: 0.7190 - val_loss: 0.1418 - val_accuracy: 0.8782
Epoch

In [19]:
model.save('model9.h5')

In [20]:
# evaluate the model
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])

Test loss: 0.12906904518604279


In [21]:
# calculate accuracy
print('Test accuracy:', score[1])

Test accuracy: 0.8182897567749023


In [ ]:
model.predict(X_test)